In [1]:
!pip install -e ./randomise-prep

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///home/users/jaemon/randomise-prep
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for randomise-prep (pyproject.toml) ... done
  Created wheel for randomise-prep: filename=randomise_prep-0.1.0-py3-none-any.whl size=4442 sha256=cb375a5d870c9c546255277df8e88cce5c8e0f01d4d7a5b0d3f3ca20f362cd84
  Stored in directory: /tmp/pip-ephem-wheel-cache-k5ckrlca/wheels/a5/7b/70/cefc8cb35db3720e41d61966b58c440bde902bbaa1e1f9e104
Successfully built randomise-prep
  Attempting uninstall: randomise-prep
    Found existing installation: randomise-prep 0.1.0
    Uninstalling randomise-prep-0.1.0:
      Successfully uninstalled randomise-prep-0.1.0

[notice] A new release of pip is available: 2

In [1]:
from randomise_prep import setup_randomise_tfce
import glob
import re
import pandas as pd
from collections import OrderedDict
from nilearn import masking
import nibabel as nib
import numpy as np

In [2]:
# Load all the questionnaire average NIFTI files
root = '/oak/stanford/groups/russpold/data/uh2/aim1'
questionnaire_avg_out = f'{root}/derivatives/output_surveyMedley_noderivs_rerun/questionnaire_averages'
bold_files = glob.glob(f'{questionnaire_avg_out}/*')

sub_ids = sorted(set([re.search('_sub_(.*).nii.gz', val).group(1) for val in bold_files]))
# exclude the subjects who contribute to the most dropout in the group mask (this was determined earlier by Jeanette)
excluded_subject_ids = ['234', '445']
sub_ids = [sub for sub in sub_ids if sub not in excluded_subject_ids]

# Load events files
events_files = sorted(glob.glob(f'{root}/BIDS/sub-s*/ses-[0-9]/func/*surveyMedley*modified*.tsv'))

# Run analysis for each questionnaire

In [3]:
def get_sub_event_file(sub_id, events_files):
    for file in events_files:
        if sub_id in file:
            return file

In [4]:
grit_questions = ["Q01", "Q02", "Q03", "Q04", "Q05", "Q06", "Q07", "Q08"]
brief_questions = ["Q09", "Q10", "Q11", "Q12", "Q13", "Q14", "Q15", "Q16", "Q17", "Q18", "Q19", "Q20", "Q21"]
future_time_questions = ["Q22", "Q23", "Q24", "Q25", "Q26", "Q27", "Q28", "Q29", "Q30", "Q31"]
upps_questions = ["Q32", "Q33", "Q34", "Q35", "Q36", "Q37"]
impulsive_venture_questions = ["Q38", "Q39", "Q40"]
questionnaires = {"grit": grit_questions, "brief": brief_questions, "future_time": future_time_questions, "upps": upps_questions, "impulsive_venture": impulsive_venture_questions}

for questionnaire in questionnaires:
    # Create a dictionary where the key is a subject who answered all questions in the questionnaire, and the value is the behavioral aggregate measure
    data_dict = OrderedDict()
    for sub_id in sub_ids:
        sub_event_file = get_sub_event_file(sub_id, events_files)
        df = pd.read_csv(sub_event_file, sep='\t')
        subset = df[df["trial_type"].isin(questionnaires[questionnaire])]
        if subset.isnull().any().any():
            continue
        else:
            avg_behavior = subset["coded_response"].mean()
            data_dict[sub_id] = avg_behavior
    
    # Create design matrix with 'centered_behavior' and 'intercept' columns
    design_matrix = pd.DataFrame()
    design_matrix["centered_behavior"] = list(data_dict.values()) - np.mean(list(data_dict.values()))
    design_matrix["intercept"] = 1
    print(design_matrix)

    contrast = {
        'behavior_slope': 'centered_behavior', 
        'overall_mean': 'intercept'
    }

    ftest = {
        'behavior_slope': ['behavior_slope'], 
        'overall_mean': ['overall_mean']
    }

    # Create NIFTI file list matching subject order in design matrix
    usable_sub_ids = [sub for sub in sub_ids if sub in data_dict]
    questionnaire_bold_files = [file for file in bold_files if questionnaire in file]
    usable_bold_files = sorted([file for file in questionnaire_bold_files if any(sub_id in file for sub_id in usable_sub_ids)])

    # Check that number of rows in design matrix matches number of bold files
    print("Number of rows in design matrix:", len(design_matrix))
    print("Number of NIFTI files:", len(usable_bold_files))

    # Make group mask
    subject_masks = glob.glob(f'{root}/derivatives/fmriprep/sub-s*/ses-['
                            f'0-9]/func/*surveyMedley*space-MNI152NLin2009cAsym*mask*.nii.gz')
    subject_masks = [mask for mask in subject_masks if any(id in mask for id in usable_sub_ids)]
    group_mask = masking.intersect_masks(subject_masks, threshold=1)
    nib.save(group_mask, f'group_mask_{questionnaire}.nii.gz')

    script_path = setup_randomise_tfce( 
        input_files=usable_bold_files,
        group_mask=f'group_mask_{questionnaire}.nii.gz',
        output_directory=f'output_glm_ftest_{questionnaire}_questionnaire',
        analysis_type='glm',
        num_perm=5000,
        design_matrix=design_matrix,
        contrast=contrast,
        ftest=ftest,
        rename_output=True,  # Rename outputs with meaningful names
    )
    print(f'Generated script: {script_path}')

        
    

    centered_behavior  intercept
0           -0.145996          1
1            0.197754          1
2            0.072754          1
3           -0.239746          1
4            0.072754          1
..                ...        ...
59          -0.145996          1
60           0.041504          1
61           0.229004          1
62          -0.020996          1
63           0.010254          1

[64 rows x 2 columns]
Number of rows in design matrix: 64
Number of NIFTI files: 64
Concatenating input files...
Created 4D file: /home/users/jaemon/output_glm_ftest_grit_questionnaire/input_data4d.nii.gz
Generated script: /home/users/jaemon/output_glm_ftest_grit_questionnaire/randomise_call.sh
    centered_behavior  intercept
0           -0.047175          1
1            0.183594          1
2            0.010517          1
3           -0.104868          1
4           -0.047175          1
..                ...        ...
59          -0.181791          1
60           0.183594          1
61        